# Big Data Project

## Configuration

In [ ]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [ ]:
//Denys: val bucketname = "unibo-bd2122-dgrushchak"
//Riccardo: val bucketname = "unibo-bd2223-rbacca"

val bucketname ="unibo-bd2122-dgrushchak"

//Denys: val path_flights_db = "s3a://"+bucketname+"/projectTest/xaa.txt"
//Denys: val path_flights_db =  "s3a://"+bucketname+"/project/small_file_10000000.txt"
//Riccardo:val path_flights_db = "s3a://"+bucketname+"/bigdata-project/part_1.txt"

val path_flights_db =  "s3a://"+bucketname+"/project/"

sc.applicationId

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

### Parsing
This parser make flights flat on base of number of the leg.

If a flight has two legs it will be splitted into 2 rows that:
* duplicate for each row the fields as: `id`, `flightDate`, `startingAirport`, `destinationAirport`, `isNonStop`, `isBasicEconomy`.
* has one value of the two that was saved together in fields as: `airplaneType`, `airlineName`, `segmentDistance`, `segmentDuration`.
* recalculate a `totalFare` for each row on base of `segmentDistance`. Sum of two rows must return the original value of `totalFare`

id is a new field that replace the orinal identifier

In [ ]:
type MyTuple = (String, String, String, String, String, Boolean, Boolean, Double, Int, Int)

case class FlatFlightData(
    id: Long,                   //1
    flightDate: String,         //2      
    startingAirport: String,    //3 
    destinationAirport: String, //4
    airplaneType: String,       //5
    airlineName: String,        //6
    isNonStop: Boolean,         //7 
    isBasicEconomy: Boolean,    //8 
    totalFare: Double,          //9
    segmentDuration: Int,     //10
    segmentDistance: Int      //11
) extends Serializable{
    // return a tuple of the object
    def un() = FlatFlightData.unapply(this).get 
}


object FlatParser{
    import scala.collection.mutable.ArrayBuffer
    import scala.collection.mutable.ListBuffer
    val commaRegex = ",(?=(?:[^\"]*\"[^\"]*\")*[^\"]*$)"
    val doubleVerticalLine = "\\|\\|"
    
    
    
    def convertAirplaneType(value: String): Array[String] = {
        //Ex. input Airbus A320||Boeing 737-800
        val result = value.split(doubleVerticalLine)
        if (result.contains("")) Array.empty
        else result
    }
    
    def convertAirlineName(value: String): Array[String] = {
        val result = value.split(doubleVerticalLine)
        if (result.contains("")) Array.empty
        else result
    }
    
    //return array of duration of each leg in minutes
    def convertSegmentsDuration(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if (result.contains("")) Array.empty
        else result.map(_.toInt/60)
    }
    
    def convertSegmentsDistance(value: String): Array[Int] = {
        val result = value.split(doubleVerticalLine)
        if(result.contains("") || result.head == "None") Array.empty
        else result.map(_.toInt)
    }
   
    def parseFlightInformationLine(line: String): Option[ListBuffer[MyTuple]] = {
        try {
            val input = line.split(commaRegex)
            
            val airplanes = convertAirplaneType(input(23)) //airplane type [String||String] 
            val airline = convertAirlineName(input(21)) // airliine name  [String||String]
            val segmentsDuration = convertSegmentsDuration(input(24)) // segments duration in seconds (but the function return duration in minutes)
            val segmentsDistance = convertSegmentsDistance(input(25))// segment distance in milles: Vector of [Int||Int]  can be [None||None]
            if (airplanes.isEmpty || 
                airline.isEmpty || 
                segmentsDuration.isEmpty || 
                segmentsDistance.isEmpty|| 
                airplanes.length != airline.length ||
                airplanes.length != segmentsDuration.length ||
                airplanes.length != segmentsDistance.length ) None
            else {
                val arr: ListBuffer[MyTuple] = ListBuffer()
                val distanceSum = segmentsDistance.reduce(_+_).toDouble
                for (i <- 0 until airplanes.length){
                    val legPriceBasedOnDistance: Double = (segmentsDistance(i).toDouble/distanceSum) * input(12).toDouble //12 baseFare in $
                    arr += ((input(2), //flightdate
                             input(3), //starting airport
                             input(4), //destination airport
                             airplanes(i),
                             airline(i),
                             input(10).toBoolean,  //isNonStop
                             input(8).toBoolean,  //isBasicEconomy
                             legPriceBasedOnDistance,
                             segmentsDuration(i),
                             segmentsDistance(i))
                    )
                }
                Some(arr)
            }
        } catch {
            case _: Exception => None
        }
    }
}

### Tolerance for duplicated data
The original dataset has many duplicated record. 
We decide to create a new dataset that has original format but have not duplicated records, then we could to use this new dataset as new main source of data for elaboration.

After elaboration of 10 GB of data we find out that only 1.2 GB of data are unique and this is less than is requested by project requirement, at this point we decided to continue using the original duplicated dataset even if it is practically not correct.

### Setup
This code:
1. Read and parse the original data
2. Create a new numeric identifier
3. Replace original textual identifier and insert the data in FlatFlighData class
4. Create a cache

In [ ]:
sc.getPersistentRDDs.foreach(_._2.unpersist()) 
import org.apache.spark.storage.StorageLevel._
//zipWithIndex trigger a spark job, for not parse twice all the data we save parsed data on the disk
val parsedData = sc.textFile(path_flights_db).flatMap(FlatParser.parseFlightInformationLine).persist(DISK_ONLY)

val rddFlightsCached = parsedData.
    zipWithIndex(). // Introduce numeric id for each flight row
    map({case (v, i) => (i, v)}).
    flatMapValues(m => m).
    map({case (k, v) => FlatFlightData.apply(k, v._1, v._2, v._3, v._4, v._5, v._6, v._7, v._8, v._9, v._10)}).persist(MEMORY_AND_DISK)
//     map({case (k, v) => FlatFlightData.apply(k, v._2, v._3, v._4, v._5, v._6, v._7, v._8, v._9, v._10, v._11)}).cache

In [ ]:
rddFlightsCached.map(_.un).take(10).foreach(println)

val numberOfRecords = rddFlightsCached.map(_.id).distinct.count
val totalNumberOfTuples = rddFlightsCached.count


def round(v: Double): Double = {
    (v*100).toInt/100.toDouble
}

## Explorative queries

1. How many distinct airports and aircraft models
2. Average travel duration for airline, only in non-stop flights
3. Percentage of basic economy tickets, based on all tickets
4. Percentage of non-stop flights (flights with one leg)
5. Average and price range of tickets
6. Average ticket price for each airline
7. Average and range of travel distance
8. Top 10 airports with more arriving flights


In [ ]:
//1.How many distinct airports and aircraft models
val distinctAirports = rddFlightsCached.
    map(x => x.startingAirport).
    distinct.
    union(rddFlightsCached.map(x => x.destinationAirport).distinct).
    distinct.count
val distinctAircraftModels = rddFlightsCached.flatMap(x => x.airplaneType).distinct.count

In [ ]:
//2. Average travel duration for each airline, in non stop flights
val distinctAirlines = rddFlightsCached.
    map(x => (x.airlineName, x.segmentDuration)).
    aggregateByKey((0.0, 0.0))((a,v)=>(a._1+v, a._2+1), (a1,a2)=>(a1._1+a2._1, a1._2+a2._2)).
    map({case(k,v)=>(k,v._1/v._2)}).
    collect().
    foreach({case (airline, value) => println(airline + " => " + round(value) + " avg minutes")})

In [ ]:
//3. Percentage of basic economy tickets, based on all tickets
round((rddFlightsCached.filter(_.isBasicEconomy).map(_.id).distinct.count.toDouble/numberOfRecords).toDouble*100) + " %"

In [ ]:
//4. Percentage of non-stop flights (flights with one leg)
round((rddFlightsCached.filter(_.isNonStop).map(_.id).distinct.count.toDouble/numberOfRecords).toDouble*100) + " %"

In [ ]:
//5. Average and price range of tickets
val ticketPrices = rddFlightsCached.map(x => (x.id,x.totalFare)).reduceByKey(_+_).map(_._2)
"Range of prices: "  + ticketPrices.min + " to " + ticketPrices.max
"Avg price: " + round((ticketPrices.sum/numberOfRecords).toDouble)

In [ ]:
//6. Average ticket price for each airline
val avgTicketPricePerAirline = rddFlightsCached.
    map(x => (x.airlineName,(x.totalFare, 1))).
    reduceByKey((a,b) => (a._1+b._1, a._2+b._2)).
    map(m => (m._1,m._2._1/m._2._2)).
    collect.foreach({case (name, value) => println(name + " => " + round(value))})

In [ ]:
//7. Average and range of travel distance
val travelDistances = rddFlightsCached.map(x => (x.id, x.segmentDistance)).reduceByKey(_+_).map(_._2)
"Range of distances: "  + travelDistances.min + " to " + travelDistances.max
"Avg travel distance: " + round((travelDistances.sum/numberOfRecords).toDouble)

In [ ]:
//8. Top 10 airports with more arriving flights
val topAirports = rddFlightsCached.
    map(x => (x.id, x.destinationAirport)).
    distinct().
    map(m => (m._2, 1)).
    reduceByKey(_+_).    
    sortBy(_._2, false).
    take(10).
    foreach({case (name, value) => println(name + " => " + value)})

## Main queries


In [ ]:
val path_output = "s3a://"+bucketname+"/spark/avgRatPerMovie" //todo

### Riccardo

> Riccardo: Aggrego su aeroporto di arrivo e di partenza, calcolando per ogni coppia un AVG dei prezzi dei biglietti. Join con dataset originale. Aggrego nuovamente sulla classificazione di prima e sulla distanza totale percorsa. Ottengo così per ogni tratta il prezzo medio e la distanza media percorsa.

In [ ]:
// Query approfondita --> Riccardo

// Prima aggregate poi Join
val result = rddFlightsCached.
    map(x => ((x.startingAirport, x.destinationAirport), (x.totalFare, 1))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v._1, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case (k,(sum,cnt)) => (k, (sum/cnt))}).
    join(rddFlightsCached.map(x => ((x.startingAirport, x.destinationAirport), x.segmentDistance)).
        aggregateByKey((0.0,0.0))((a,v)=>(a._1+v, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
        map({case (k,(sum,cnt)) => (k, (sum/cnt))})).
    sortByKey().
    take(10).foreach(println)


// Prima Join poi aggregate
/*val result = rddFlightsCached.
    map(x => ((x.startingAirport, x.destinationAirport), (x.totalFare, 1))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+v._1, a._2+1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2)).
    map({case (k,(sum,cnt)) => (k, (sum/cnt))}).
    join(rddFlightsCached.map(x => ((x.startingAirport, x.destinationAirport), x.segmentDistance))). //([startingAirport, arrivingAirport], [prezzo medio, distanza percorsa])
    aggregateByKey((0.0,0.0,0.0))((a,v)=>(a._1+v._2, a._2+1, v._1),(a1,a2)=>(a1._1+a2._1,a1._2+a2._2,a1._2)).
    map({case (k,(sum,cnt,avg)) => (k, (sum/cnt,avg))}).
    map({case (k,(v1,v2)) => (k, (v2,v1))}).
    sortByKey().
    take(10).foreach(println)*/




### Denys
I will calculate the velocity of each airplane type
> Denys: aggrego su “airplane model” per calcolare la “segment distance” totale percorsa da ogni modello, poi faccio self-join e aggregazione per determinare il “travel duration” per ogni “airplane model”. Alla fine determino la velocità media di ogni modello partendo dai dati aggregati. Visualizzo 10 aerei con la velocità media più alta

In [ ]:
//return velocity in miles per hour
def getVelocity(distance: Int, durationInMinutes: Int): Double = {
    round(distance.toDouble/(durationInMinutes.toDouble/60))
}

First solution is exactly as in the problem description: agregation two values separately and then join:

In [ ]:
val totalAirplanesDistance = rddFlightsCached.map(m => (m.airplaneType, m.segmentDistance)).reduceByKey(_+_)
val totalAirplanesDuration = rddFlightsCached.map(m => (m.airplaneType, m.segmentDuration)).reduceByKey(_+_)

totalAirplanesDistance.join(totalAirplanesDuration).
    map({case (name, (dis, dur)) =>  (name, getVelocity(dis, dur))}).
    sortBy(_._2,false).
    take(10).
    foreach({case (a, v) => println(a + " ----> " + v + " mph")})

In next implementation I tried to make a repartitioning of the data on base of common key but it gives a worst result because repartioning overhead is much bigger than suffle in the first query.

For example: for 7 GB dataset the first query have only ~1Mb of suffle reads and writes while this solution have ~460MB of shuffle writes and ~920MB of shuffle reads

In [ ]:
val repartitionedValues = rddFlightsCached.map(m => (m.airplaneType, (m.segmentDistance, m.segmentDuration))).repartition(12)

val totalAirplanesDistance = repartitionedValues.map(m => (m._1, m._2._1)).reduceByKey(_+_)
val totalAirplanesDuration = repartitionedValues.map(m => (m._1, m._2._2)).reduceByKey(_+_)

totalAirplanesDistance.join(totalAirplanesDuration).
    map({case (name, (dis, dur)) =>  (name, getVelocity(dis, dur))}).
    sortBy(_._2,false).
    take(10).
    foreach({case (a, v) => println(a + " ----> " + v + " mph")})

The fastes way for the velocity calculation is aggregation of two field together. This approach allows not join the data. 

In [ ]:
rddFlightsCached.
    map(m => (m.airplaneType, (m.segmentDistance, m.segmentDuration))).
    reduceByKey({case ((dis1, dur1), (dis2, dur2)) => (dis1+dis2, dur1+dur2)}).
    map({case (name, (dis, dur)) =>  (name, getVelocity(dis, dur))}).
    sortBy(_._2, false).
    take(10).
    foreach({case (a, v) => println(a + " ----> " + v + " mph")})

Alternative velocity valutation: first we calculate medium velocity of each flight and then we calculate the madium velocity for each airplane type. Obviously the result is different

In [ ]:
rddFlightsCached.
    map(m => (m.airplaneType, (m.segmentDistance.toDouble, m.segmentDuration.toDouble/60))).
    aggregateByKey((0.0,0.0))((a,v)=>(a._1+(v._1/v._2), a._2+1),(a1,a2)=>(a1._1+a2._1, a1._2+a2._2)).
    map({case (name, (velocitySum, count)) =>  (name, round(velocitySum/count))}).
    sortBy(_._2, false).
    take(10).
    foreach({case (a, v) => println(a + " ----> " + v + " mph")})